In [1]:
import numpy as np
import pandas as pd
import string

In [2]:
nfeatures    = 214
nbits        = 9
v_sup_width  = 120 # number of valence support vectors
a_sup_width  = 155 # number of arousal support vectors
rom_depth    = 1024
entire_width = 2048 # 16 1024x128 ROMs

In [3]:
v_support_df = pd.read_csv("RFE_{nf}_nbits_{nb}_v_support.txt".format(nf=nfeatures, nb=nbits), header=None)
a_support_df = pd.read_csv("RFE_{nf}_nbits_{nb}_a_support.txt".format(nf=nfeatures, nb=nbits), header=None)
v_alpha_df   = pd.read_csv("RFE_{nf}_nbits_{nb}_v_alpha_vector.txt".format(nf=nfeatures, nb=nbits), header=None)
a_alpha_df   = pd.read_csv("RFE_{nf}_nbits_{nb}_a_alpha_vector.txt".format(nf=nfeatures, nb=nbits), header=None)
v_features_df = pd.read_csv("RFE_{nf}_nbits_{nb}_v_features.txt".format(nf=nfeatures, nb=nbits), header=None)
a_features_df = pd.read_csv("RFE_{nf}_nbits_{nb}_a_features.txt".format(nf=nfeatures, nb=nbits), header=None)

In [4]:
v_support_memory = [[0]*nfeatures]*v_sup_width
a_support_memory = [[0]*nfeatures]*a_sup_width

for i in range(v_sup_width):
    v_support_memory[i] = v_support_df[i*nfeatures : i*nfeatures+nfeatures].squeeze().tolist()
    
for i in range(a_sup_width):
    a_support_memory[i] = a_support_df[i*nfeatures : i*nfeatures+nfeatures].squeeze().tolist()
    
v_alpha_memory = v_alpha_df.squeeze().tolist()
a_alpha_memory = a_alpha_df.squeeze().tolist()

In [5]:
# Top diagonals
def diagonalize_array(array):
    num_rows = len(array)
    num_cols = len(array[0])
    
    diagonalized = []
    
    for k in range(0, num_cols + num_rows - 1):
        curr_row = []
        for j in range(0, k + 1):
            i  = k - j
            if (i < num_rows and j < num_cols):
                curr_row.append(array[i][j])
                
        # Append 0's to curr_row
        if (k < num_rows):
            curr_row = curr_row + [0]*(num_cols - len(curr_row)) # only append 0's after the payload
        else:
            curr_row = [0]*(k - num_rows + 1) + curr_row # append a certain # of 0's before the payload
            curr_row = curr_row + [0]*(num_cols - len(curr_row)) # adjust how many 0's to add after the payload
            
        diagonalized.append(curr_row)

    return diagonalized

In [6]:
if nfeatures == 1:
    v_sup_diag = [[x] for x in v_support_memory]
    a_sup_diag = [[x] for x in a_support_memory]
else:
    v_sup_diag = np.array(diagonalize_array(v_support_memory))
    a_sup_diag = np.array(diagonalize_array(a_support_memory))

In [7]:
def binarize_9bit_signed(num):
    bin_rep = bin(num & 0b111111111)[2:]
    return '0'*(9-len(bin_rep)) + bin_rep

In [8]:
v_support_bin = []
a_support_bin = []
v_alpha_bin   = []
a_alpha_bin   = []

for i in range(nfeatures + v_sup_width - 1):
    curr_row = ''.join(list(map(binarize_9bit_signed, v_sup_diag[i])))
    v_support_bin.append(curr_row)
    
for i in range(nfeatures + a_sup_width - 1):
    curr_row = ''.join(list(map(binarize_9bit_signed, a_sup_diag[i])))
    a_support_bin.append(curr_row)
    
for i in range(v_sup_width):
    curr_row = binarize_9bit_signed(v_alpha_memory[i])
    v_alpha_bin.append(curr_row)
    
for i in range(a_sup_width):
    curr_row = binarize_9bit_signed(a_alpha_memory[i])
    a_alpha_bin.append(curr_row)

In [9]:
entire_memory = []

for i in range(nfeatures + v_sup_width - 1):
    curr_v_sup_row = v_support_bin[i] + '0'*(entire_width - len(v_support_bin[i]))
    entire_memory.append(curr_v_sup_row)
    
for i in range(nfeatures + a_sup_width - 1):
    curr_a_sup_row = a_support_bin[i] + '0'*(entire_width - len(a_support_bin[i]))
    entire_memory.append(curr_a_sup_row)
    
for i in range(v_sup_width):
    curr_v_alpha_row = v_alpha_bin[i] + '0'*(entire_width - len(v_alpha_bin[i]))
    entire_memory.append(curr_v_alpha_row)
    
for i in range(a_sup_width):
    curr_a_alpha_row = a_alpha_bin[i] + '0'*(entire_width - len(a_alpha_bin[i]))
    entire_memory.append(curr_a_alpha_row)
    
for i in range(rom_depth - ((nfeatures + v_sup_width - 1) + (nfeatures + a_sup_width - 1) + v_sup_width + a_sup_width)):
    entire_memory.append('0'*entire_width)

In [10]:
entire_memory_hex0 = []
entire_memory_hex1 = []
entire_memory_hex2 = []
entire_memory_hex3 = []
entire_memory_hex4 = []
entire_memory_hex5 = []
entire_memory_hex6 = []
entire_memory_hex7 = []
entire_memory_hex8 = []
entire_memory_hex9 = []
entire_memory_hex10 = []
entire_memory_hex11 = []
entire_memory_hex12 = []
entire_memory_hex13 = []
entire_memory_hex14 = []
entire_memory_hex15 = []

for i in range(rom_depth):
    entire_string = hex(int(entire_memory[i], 2))[2:].zfill(entire_width//4)
    entire_memory_hex0.append(entire_string[0*32:1*32])
    entire_memory_hex1.append(entire_string[1*32:2*32])
    entire_memory_hex2.append(entire_string[2*32:3*32])
    entire_memory_hex3.append(entire_string[3*32:4*32])
    entire_memory_hex4.append(entire_string[4*32:5*32])
    entire_memory_hex5.append(entire_string[5*32:6*32])
    entire_memory_hex6.append(entire_string[6*32:7*32])
    entire_memory_hex7.append(entire_string[7*32:8*32])
    entire_memory_hex8.append(entire_string[8*32:9*32])
    entire_memory_hex9.append(entire_string[9*32:10*32])
    entire_memory_hex10.append(entire_string[10*32:11*32])
    entire_memory_hex11.append(entire_string[11*32:12*32])
    entire_memory_hex12.append(entire_string[12*32:13*32])
    entire_memory_hex13.append(entire_string[13*32:14*32])
    entire_memory_hex14.append(entire_string[14*32:15*32])
    entire_memory_hex15.append(entire_string[15*32:16*32])

In [14]:
f = open('tech-tstech28-cache-local/imem0.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex0[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem0.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem1.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex1[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem1.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem2.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex2[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem2.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem3.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex3[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem3.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem4.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex4[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem4.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem5.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex5[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem5.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem6.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex6[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem6.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem7.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex7[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem7.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem8.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex8[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem8.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem9.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex9[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem9.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem10.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex10[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem10.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem11.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex11[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem11.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem12.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex12[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem12.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem13.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex13[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem13.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem14.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex14[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem14.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem15.hex', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:9] + str.upper(entire_memory_hex15[i]) + lines[i][41:44])
write_lines.append(lines[rom_depth])
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem15.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

In [15]:
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem0_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex0)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem1_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex1)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem2_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex2)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem3_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex3)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem4_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex4)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem5_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex5)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem6_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex6)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem7_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex7)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem8_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex8)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem9_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex9)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem10_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex10)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem11_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex11)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem12_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex12)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem13_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex13)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem14_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex14)))
f.close()

f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem15_raw.hex'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(list(map(lambda x: x + '\n', entire_memory_hex15)))
f.close()

In [16]:
f = open('tech-tstech28-cache-local/imem0.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex0[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem0.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem1.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex1[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem1.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem2.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex2[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem2.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem3.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex3[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem3.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem4.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex4[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem4.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem5.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex5[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem5.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem6.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex6[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem6.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem7.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex7[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem7.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem8.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex8[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem8.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem9.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex9[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem9.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem10.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex10[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem10.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem11.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex11[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem11.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem12.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex12[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem12.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem13.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex13[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem13.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem14.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex14[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem14.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()

f = open('tech-tstech28-cache-local/imem15.rom.v', 'r')
lines = f.readlines()
write_lines = []
for i in range(rom_depth):
    write_lines.append(lines[i][0:10] + entire_memory_hex15[i] + '\n')
f = open('tech-tstech28-cache-new_RFE_{nf}_nbits_{nb}/imem15.rom.v'.format(nf=nfeatures, nb=nbits), 'w')
f.writelines(write_lines)
f.close()